In [1]:
import os
file_names:list[str] = os.listdir('stations')
abs_path:list[str] = []
for name in file_names:
    if '每日各站進出站人數' in name:
        abs_path.append(os.path.abspath(f'stations/{name}')) #抓取檔案位置的絕對路徑

abs_path

['/workspaces/2024_02_20_2_4/lesson22/stations/每日各站進出站人數2022.csv',
 '/workspaces/2024_02_20_2_4/lesson22/stations/每日各站進出站人數2023.csv',
 '/workspaces/2024_02_20_2_4/lesson22/stations/每日各站進出站人數20190423-20191231.csv',
 '/workspaces/2024_02_20_2_4/lesson22/stations/每日各站進出站人數2021.csv',
 '/workspaces/2024_02_20_2_4/lesson22/stations/每日各站進出站人數2020.csv']

In [2]:
import pandas as pd
from pandas import DataFrame

all_df:list[DataFrame] = [pd.read_csv(path) for path in abs_path]
display(all_df)

[       trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20220101      900             8645             8526
 1       20220101      910              996             1243
 2       20220101      920             1369             1711
 3       20220101      930             3663             4535
 4       20220101      940             1532             1845
 ...          ...      ...              ...              ...
 86547   20221231     7360             3984             3893
 86548   20221231     7361               92              115
 86549   20221231     7362              131              159
 86550   20221231     7380              494              426
 86551   20221231     7390              313              274
 
 [86552 rows x 4 columns],
        trnOpDate  staCode  gateInComingCnt  gateOutGoingCnt
 0       20230101      900             8030             8241
 1       20230101      910             1017             1326
 2       20230101      920             1400             

In [3]:
inOut_df = pd.concat(all_df) #利用concat將5個檔案整合連結成一個
inOut_df.columns = ['乘車日', '車站代碼', '進站人數', '出站人數']
inOut_df

,乘車日,車站代碼,進站人數,出站人數
0,20220101,900,8645,8526
1,20220101,910,996,1243
2,20220101,920,1369,1711
3,20220101,930,3663,4535
4,20220101,940,1532,1845
...,...,...,...,...
86927,20201231,7360,3523,3602
86928,20201231,7361,3,18
86929,20201231,7362,28,34
86930,20201231,7380,609,604


In [4]:
import requests
import pandas as pd
#車站基本資料集 - https://data.gov.tw/dataset/33425
url2 = 'https://ods.railway.gov.tw/tra-ods-web/ods/download/dataResource/0518b833e8964d53bfea3f7691aea0ee'
response = requests.get(url2)
if response.status_code == 200:
    station_data = response.json()

station_df = pd.DataFrame(station_data,columns=['stationCode','stationName'])
station_df.columns = ['車站代碼','中文站名']
station_df['車站代碼'] = station_df['車站代碼'].astype(int)
station_df

,車站代碼,中文站名
0,900,基隆
1,910,三坑
2,920,八堵
3,930,七堵
4,940,百福
...,...,...
238,7360,瑞芳
239,7361,海科館
240,7362,八斗子
241,7380,四腳亭


In [5]:
station_all_df = pd.merge(inOut_df, station_df, on='車站代碼', how='left')
station_all_df

,乘車日,車站代碼,進站人數,出站人數,中文站名
0,20220101,900,8645,8526,基隆
1,20220101,910,996,1243,三坑
2,20220101,920,1369,1711,八堵
3,20220101,930,3663,4535,七堵
4,20220101,940,1532,1845,百福
...,...,...,...,...,...
406756,20201231,7360,3523,3602,瑞芳
406757,20201231,7361,3,18,海科館
406758,20201231,7362,28,34,八斗子
406759,20201231,7380,609,604,四腳亭


In [6]:
station_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [7]:
#移除內容值為 Nan的欄位
station_all_df1 = station_all_df.dropna()
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   乘車日     406761 non-null  int64 
 1   車站代碼    406761 non-null  int64 
 2   進站人數    406761 non-null  int64 
 3   出站人數    406761 non-null  int64 
 4   中文站名    406761 non-null  object
dtypes: int64(4), object(1)
memory usage: 15.5+ MB


In [8]:
#將乘車日改為Datatime
station_all_df1['乘車日'] = pd.to_datetime(station_all_df1['乘車日'].astype(str)) #轉datatime時，astype可有可無
station_all_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406761 entries, 0 to 406760
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站代碼    406761 non-null  int64         
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
 4   中文站名    406761 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 15.5+ MB


In [13]:
station_all_df2 = station_all_df1.sort_values(by='乘車日') #sort_values做昇序排列
station_all_df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 173161 to 172948
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站代碼    406761 non-null  int64         
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
 4   中文站名    406761 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 18.6+ MB


In [15]:
#dt是series資料轉換成daytime
station_all_df2['乘車日'].dt.strftime('%Y-%m')

173161    2019-04
173314    2019-04
173315    2019-04
173316    2019-04
173317    2019-04
           ...   
173021    2023-12
173022    2023-12
173023    2023-12
173011    2023-12
172948    2023-12
Name: 乘車日, Length: 406761, dtype: object

In [16]:
station_all_df2

,乘車日,車站代碼,進站人數,出站人數,中文站名
173161,2019-04-23,900,8442,7743,基隆
173314,2019-04-23,5080,15,14,鎮安
173315,2019-04-23,5090,368,325,林邊
173316,2019-04-23,5100,95,95,佳冬
173317,2019-04-23,5110,66,50,東海
...,...,...,...,...,...
173021,2023-12-31,3450,1063,869,林內
173022,2023-12-31,3460,325,269,石榴
173023,2023-12-31,3470,6964,6271,斗六
173011,2023-12-31,3400,693,677,永靖


In [17]:
station_all_df2.groupby('中文站名')[['進站人數','出站人數']].agg(['sum','max','min'])

進站人數                出站人數           
          sum   max  min      sum   max  min
中文站名                                        
七堵    9044563  7704  141  9115329  8288  210
三坑    2080517  1715   34  2140894  1835   31
三塊厝    786111   933   16   824381  1013   15
三姓橋   1760760  2125    3  1594509  1603   16
三民      12535    52    0    18652    57    0
...       ...   ...  ...      ...   ...  ...
鼓山     469414  2352    9   476151  1055   16
龍井     596686  2131   36   589722  5506   35
龍泉      24965   103    0    30795   390    1
龍港      30146   106    2    38335   130    2
龜山     141198   389    3   134504   287    0

[241 rows x 6 columns]

In [18]:
station_all_df2.groupby(['中文站名',station_all_df2['乘車日'].dt.year])[['進站人數','出站人數']].agg(['sum','max','min'])

進站人數                 出站人數            
               sum   max   min      sum   max   min
中文站名 乘車日                                           
七堵   2019  1512951  7417  2103  1560506  8078  2438
     2020  2023475  7366  2616  2055442  8288  2607
     2021  1619233  7314   614  1594220  7702   630
     2022  1764417  7280   141  1776557  7289   210
     2023  2124487  7704  1472  2128604  7754  1459
...            ...   ...   ...      ...   ...   ...
龜山   2019    26408   199    49    25265   166    48
     2020    33665   209    43    32326   206    30
     2021    23193   194     9    22534   157     9
     2022    27966   389     3    24766   178     0
     2023    29966   255    31    29613   287    33

[1198 rows x 6 columns]

In [20]:
df3 = station_all_df2.groupby(['中文站名',station_all_df2['乘車日'].dt.strftime('%Y-%m')])[['進站人數','出站人數']].agg(['sum','max','min'])
df3

進站人數                出站人數            
                 sum   max   min     sum   max   min
中文站名 乘車日                                            
七堵   2019-04   46711  6635  4616   47817  7042  4552
     2019-05  186400  7075  4599  190985  7568  4533
     2019-06  173853  7141  4150  177242  7319  4242
     2019-07  182569  6791  4433  190198  7482  4652
     2019-08  172647  6898  2377  178678  7202  2500
...              ...   ...   ...     ...   ...   ...
龜山   2023-08    2318   121    45    2378   117    50
     2023-09    2398   126    46    2393   135    42
     2023-10    2660   128    31    2669   160    38
     2023-11    2563   110    48    2594   137    49
     2023-12    3091   255    48    2794   287    48

[13619 rows x 6 columns]

In [23]:
df3.columns.names = ['人數','統計']
df3

人數              進站人數                出站人數            
統計               sum   max   min     sum   max   min
中文站名 乘車日                                            
七堵   2019-04   46711  6635  4616   47817  7042  4552
     2019-05  186400  7075  4599  190985  7568  4533
     2019-06  173853  7141  4150  177242  7319  4242
     2019-07  182569  6791  4433  190198  7482  4652
     2019-08  172647  6898  2377  178678  7202  2500
...              ...   ...   ...     ...   ...   ...
龜山   2023-08    2318   121    45    2378   117    50
     2023-09    2398   126    46    2393   135    42
     2023-10    2660   128    31    2669   160    38
     2023-11    2563   110    48    2594   137    49
     2023-12    3091   255    48    2794   287    48

[13619 rows x 6 columns]

In [24]:
#樞紐分析的動作
df3.stack(level=['人數','統計']).unstack(level=['中文站名','統計'])

/tmp/ipykernel_10574/1981058358.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df3.stack(level=['人數','統計']).unstack(level=['中文站名','統計'])


中文站名                七堵                       三坑                      三塊厝  \
統計                 sum     max     min      sum     max     min      sum   
乘車日     人數                                                                 
2019-04 出站人數   47817.0  7042.0  4552.0  10838.0  1493.0  1102.0   3182.0   
        進站人數   46711.0  6635.0  4616.0  10747.0  1432.0  1139.0   2979.0   
2019-05 出站人數  190985.0  7568.0  4533.0  42267.0  1629.0  1072.0  12054.0   
        進站人數  186400.0  7075.0  4599.0  41404.0  1573.0  1047.0  11309.0   
2019-06 出站人數  177242.0  7319.0  4242.0  40133.0  1552.0  1028.0  11482.0   
...                ...     ...     ...      ...     ...     ...      ...   
2023-10 進站人數  192253.0  7377.0  4702.0  45048.0  1651.0  1188.0  19190.0   
2023-11 出站人數  194509.0  7604.0  4532.0  47204.0  1803.0  1131.0  20464.0   
        進站人數  194962.0  7576.0  4506.0  44315.0  1681.0  1106.0  19235.0   
2023-12 出站人數  195570.0  7741.0  4476.0  48784.0  1835.0  1141.0  21503.0   
        進站人數  196282.0  7704.0  4448.0  46271.0  1715.0  1145.0  20191.0   

中文站名                            三姓橋  ...     龍井      龍泉                  龍港  \
統計              max    min      sum  ...    min     sum    max   min    sum   
乘車日     人數                           ...                                      
2019-04 出站人數  485.0  319.0   8651.0  ...  351.0   150.0   30.0   7.0  193.0   
        進站人數  416.0  312.0   9768.0  ...  350.0   120.0   30.0   8.0  193.0   
2019-05 出站人數  470.0  327.0  32514.0  ...  265.0   441.0   39.0   4.0  661.0   
        進站人數  410.0  302.0  36784.0  ...  284.0   421.0   27.0   4.0  608.0   
2019-06 出站人數  498.0  299.0  30549.0  ...  284.0   538.0   58.0   5.0  644.0   
...             ...    ...      ...  ...    ...     ...    ...   ...    ...   
2023-10 進站人數  933.0  172.0  41165.0  ...  126.0   571.0   33.0   5.0  839.0   
2023-11 出站人數  790.0  571.0  39833.0  ...  345.0   622.0   49.0   7.0  894.0   
        進站人數  694.0  594.0  43417.0  ...  368.0   541.0   47.0   6.0  872.0   
2023-12 出站人數  854.0  599.0  40425.0  ...  360.0  1471.0  271.0  12.0  855.0   
        進站人數  761.0  571.0  44781.0  ...  347.0   622.0   48.0   8.0  946.0   

中文站名                          龜山               
統計             max   min     sum    max   min  
乘車日     人數                                     
2019-04 出站人數  32.0  18.0   828.0  160.0  65.0  
        進站人數  35.0  15.0   852.0  182.0  75.0  
2019-05 出站人數  32.0  14.0  3247.0  146.0  73.0  
        進站人數  32.0   9.0  3366.0  157.0  79.0  
2019-06 出站人數  35.0  11.0  2976.0  146.0  61.0  
...            ...   ...     ...    ...   ...  
2023-10 進站人數  42.0  11.0  2660.0  128.0  31.0  
2023-11 出站人數  45.0  19.0  2594.0  137.0  49.0  
        進站人數  49.0  17.0  2563.0  110.0  48.0  
2023-12 出站人數  61.0  15.0  2794.0  287.0  48.0  
        進站人數  87.0  15.0  3091.0  255.0  48.0  

[114 rows x 723 columns]